<a href="https://colab.research.google.com/github/cras-lab/Finance1/blob/main/%EC%8B%9C%EA%B3%84%EC%97%B4_%EA%B5%AC%EC%84%B1%EC%9A%94%EC%86%8C%EC%99%80_1%EC%B0%A8%EC%B0%A8%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

아래코드는 선택적이며, Colab에서 한글을 출력하고 싶을때만 사용한다.
한글폰트를 설치한다. <BR>
설치가 끝나면 반드시 <런타임>메뉴에서   "런타임 다시 시작"을 수행해야 변경이 반영된다.

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

시계열 분석을 위해 yfinance를 설치한다.

In [2]:
pip install -q yfinance

필요한 모듈을 import 한다.

In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

야후 finance의 Open API를 사용하여, 삼성전자 주가를 불러 온다.

In [4]:
plt.rc('font', family='NanumBarunGothic')
Tickers = [ '^KS11', '^GSPC']
start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2020, 1, 1)
df = yf.download(Tickers, start=start, end=end, interval='1mo').Close
Prices = [ df['^GSPC'], df[ '^KS11']]

[*********************100%***********************]  2 of 2 completed


시계열의 각 구성요소를 시각화 해 본다.

In [ ]:
plt.figure(figsize=(12, 24))
seasonal_decompose(Prices[0], period=12).plot()
seasonal_decompose(Prices[1], period=12).plot()
plt.show()

추세: 시작점과 끝점이 다르거나, 시계열에서 상승/하향 기울기를 가질때 나타난다.
S&P와 KOSPI가 각각 어떤 추세를 가지는지 살펴보자.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(Prices[0])
plt.plot(Prices[1])
plt.title('S&P .vs. KOSPI 가격비교')
plt.ylabel('지수')
plt.xlabel('날짜')
plt.show()

자기상관과 부분자기상관 관계를 살펴보면, 현재 값과 과거 값 사이의 종속성을 볼 수 있다.추세가 있다고 반드시 자기상관이 생기는 것은 아니지만, 자기 상관이 있을 개연성이 높다.

In [ ]:
sm.graphics.tsa.plot_acf(Prices[0], lags=30)
sm.graphics.tsa.plot_acf(Prices[1], lags=30)
plt.xlabel('Number of Lags')
plt.show()

부분 자기상관 관계를 살펴보자.
부분 상관계수는 다른 변수들의 영향을 제거하고 두 변수가 다른 변수들로부터 독립적으로 얼마나 관련성을 가지는지의 지표

EX) A, B가 양의 상관관계를 가지고 , C가 A,B 모두와 양의 상관관계를 가진다고 가정할 때, C의 영향을 제거하지 않으면, A와 B의 상관관계는 실제보다 높게 나타날 수 있다. A와 B 사이의 부분상관관계수를 계산하면, C의 영향을 제거하고 A와 B 사이의 상관관계를 보다 정확하게 파악할 수 있다.
일반적으로는 Yule-Walker 방정식이나 Durbin-Levinson 알고리즘 등을 사용한다. 

In [ ]:
sm.graphics.tsa.plot_pacf(Prices[0], lags=30)
sm.graphics.tsa.plot_pacf(Prices[1], lags=30)
plt.xlabel('Number of Lags')
plt.show()

차분을 하는 것 만으로도 정상성을 어느 정도 확보할 수 있다.1차 차분을 한 다음, 그래프 부터 각종 지표를 비교해 보자.

In [ ]:
df_diff = df.diff()
diff_Prices = [ df_diff[ '^GSPC'] , df_diff['^KS11'] ]
plt.figure(figsize=(10, 6))
plt.plot(diff_Prices[0])
plt.plot(diff_Prices[1])
plt.title('Differenced S&P-500 & KOSPI Price')
plt.ylabel('Index')
plt.xlabel('Date')
plt.show()

차분한 값들의 자기상관 그래프를 그려보자.

In [ ]:
sm.graphics.tsa.plot_acf(diff_Prices[0].dropna(),lags=30)
sm.graphics.tsa.plot_acf(diff_Prices[1].dropna(),lags=30)
plt.xlabel('Number of Lags')
plt.show()

차분한 값들의 부분 자기상관관계도 그래프로 그려보자.

In [ ]:
sm.graphics.tsa.plot_pacf(diff_Prices[0].dropna() , lags=30)
sm.graphics.tsa.plot_pacf(diff_Prices[1].dropna() , lags=30)
#plt.xlabel('Number of Lags')
plt.xlabel('지연수')
plt.show()

ADF(Augmented Dickey-Fuller) 검정을 해보면, 정상성에 대한 보다 정확한 결과를 알 수 있다.
신뢰구간 95%검정의 경우, P 값이 0.05보다 작으면, "단위근이 존재한다."는 귀무가설을 기각하고, 정상성이 있음을 확인해 준다.
정상성을 가질수록 검정 통계량 값도 작아진다.
차분을 취하기 전후의 두 값을 비교해 보자.

In [ ]:
stat_test = adfuller(Prices[0].dropna())[0:2] 
print("차분 전 S&P의 ADF 통계검정량과 p-값: {}".format(stat_test))

stat_test = adfuller(diff_Prices[0].dropna())[0:2] 
#print("S&P:: The test statistic and p-value of ADF test after differencing are {}".format(stat_test1))
print("차분 후 S&P의 ADF 통계검정량과 p-값: {}".format(stat_test))

stat_test = adfuller(Prices[1].dropna())[0:2] 
#print("S&P:: The test statistic and p-value of ADF test after differencing are {}".format(stat_test1))
print("차분 전 KOSPI의 ADF 통계검정량과 p-값: {}".format(stat_test))
stat_test = adfuller(diff_Prices[1].dropna())[0:2] 
#print("KOSPI:: The test statistic and p-value of ADF test after differencing are {}".format(stat_test2))
print("차분 후 KOSPI의 ADF 통계검정량과 p-값: {}".format(stat_test))